In [ ]:
# !gdown --id 19dk0F5Rn29mMV18heRHELAW1pw_NzJoE -O dataset.zip

In [ ]:
# !unzip -q dataset.zip -d dataset

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2

In [ ]:
base_model = MobileNetV2(
    input_shape = (224, 224, 3), # standard input size
    include_top = False,   # Remove final classification layer
    weights = "imagenet"    # Load pretrained imagenet weights
)

In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_224"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,257,984 (8.61 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from tensorflow.keras import layers, models

# Build the model
model1 = models.Sequential([

    # Base model (pretrained, e.g., MobileNetV2)
    # This part extracts features from the input images
    base_model,

    # Convert the 3D feature maps from the base model into a 1D vector
    # GlobalAveragePooling2D averages each feature map into a single number
    layers.GlobalAveragePooling2D(),

    # Fully connected layer with 128 neurons
    # Learns patterns and relationships from the features
    layers.Dense(128, activation='relu'),

    # Dropout layer to prevent overfitting
    # Randomly turns off 30% of neurons during training
    layers.Dropout(0.3),

    # Output layer with 51 neurons (one for each class)
    # Softmax activation gives probability for each class
    layers.Dense(51, activation='softmax')
])

In [ ]:
model1.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 51)             │         6,579 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,428,531 (9.26 MB)

 Trainable params: 170,547 (666.20 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import tensorflow as tf

# ------------------------ Parameters ------------------------
batch_size = 64             # Number of images per batch
image_size = (224, 224)      # Input size for MobileNetV2
dataset_dir = r'/content/dataset/dataset/'  # Path to your dataset

# ------------------------ Load datasets ------------------------

# Load training images from folder
# Images are resized to (224,224) and grouped in batches of 32
train_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir + "train",
    seed=42,            # Makes shuffling reproducible
    image_size=image_size,
    batch_size=batch_size
)

# Load validation images (used to check model performance during training)
val_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir + "val",
    seed=777,           # Different seed for validation (doesn't need to match train)
    image_size=image_size,
    batch_size=batch_size
)

# Load test images (used to evaluate the model at the end)
test_ds = tf.keras.utils.image_dataset_from_directory(
    dataset_dir + "test",
    seed=42,
    image_size=image_size,
    batch_size=batch_size
)

# ------------------------ Preprocess images ------------------------
# MobileNetV2 expects input images to be normalized in a specific way
# This step applies that normalization
train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds   = val_ds.map(lambda x, y: (preprocess_input(x), y))
test_ds  = test_ds.map(lambda x, y: (preprocess_input(x), y))

# ------------------------ Optimize performance ------------------------
AUTOTUNE = tf.data.AUTOTUNE

# Shuffle training dataset with small buffer
train_ds = train_ds.shuffle(buffer_size=100)

# Prefetch batches for smoother training
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

Found 69953 files belonging to 51 classes.
Found 15226 files belonging to 51 classes.
Found 15234 files belonging to 51 classes.


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Freeze base model
base_model.trainable = False

# Compile
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = EarlyStopping(
    monitor='val_loss',       # metric to watch (validation loss)
    patience=3,               # how many epochs to wait for improvement
    restore_best_weights=True, # after stopping, restore the weights with best val_loss
    verbose=1                 # print messages when stopping
)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',  # metric to watch
    factor=0.5,          # multiply learning rate by 0.5 if plateau occurs
    patience=2,          # wait 2 epochs before reducing LR
    verbose=1            # print messages when LR changes
)

callbacks = [early_stop, reduce_lr]

# Train
history_phase1 = model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 208s 158ms/step - accuracy: 0.6745 - loss: 1.1884 - val_accuracy: 0.9060 - val_loss: 0.2903 - learning_rate: 0.0010
Epoch 2/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 203s 121ms/step - accuracy: 0.8998 - loss: 0.3076 - val_accuracy: 0.9396 - val_loss: 0.1733 - learning_rate: 0.0010
Epoch 3/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 155s 134ms/step - accuracy: 0.9293 - loss: 0.2077 - val_accuracy: 0.9494 - val_loss: 0.1541 - learning_rate: 0.0010
Epoch 4/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 152s 132ms/step - accuracy: 0.9385 - loss: 0.1767 - val_accuracy: 0.9543 - val_loss: 0.1326 - learning_rate: 0.0010
Epoch 5/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 166s 142ms/step - accuracy: 0.9490 - loss: 0.1486 - val_accuracy: 0.9557 - val_loss: 0.1308 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 5.


In [ ]:
# Unfreeze top 50 layers
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Compile with smaller LR
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train
history_phase2 = model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 197s 152ms/step - accuracy: 0.8894 - loss: 0.3735 - val_accuracy: 0.9565 - val_loss: 0.1245 - learning_rate: 5.0000e-05
Epoch 2/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 155s 132ms/step - accuracy: 0.9678 - loss: 0.0944 - val_accuracy: 0.9678 - val_loss: 0.0973 - learning_rate: 5.0000e-05
Epoch 3/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 156s 132ms/step - accuracy: 0.9808 - loss: 0.0540 - val_accuracy: 0.9735 - val_loss: 0.0845 - learning_rate: 5.0000e-05
Epoch 4/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 170s 145ms/step - accuracy: 0.9852 - loss: 0.0431 - val_accuracy: 0.9739 - val_loss: 0.0862 - learning_rate: 5.0000e-05
Epoch 5/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 186s 130ms/step - accuracy: 0.9902 - loss: 0.0318 - val_accuracy: 0.9787 - val_loss: 0.0631 - learning_rate: 5.0000e-05
Restoring model weights from the end of the best epoch: 5.


In [ ]:
# Unfreeze all layers
base_model.trainable = True

# Optional: freeze first 100 layers to keep low-level features
for layer in base_model.layers[:100]:
    layer.trainable = False

# Compile with very small LR
model1.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train
history_phase3 = model1.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=callbacks
)

Epoch 1/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 213s 161ms/step - accuracy: 0.9796 - loss: 0.0612 - val_accuracy: 0.9804 - val_loss: 0.0619 - learning_rate: 1.0000e-05
Epoch 2/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 156s 132ms/step - accuracy: 0.9894 - loss: 0.0303 - val_accuracy: 0.9813 - val_loss: 0.0600 - learning_rate: 1.0000e-05
Epoch 3/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 161s 137ms/step - accuracy: 0.9925 - loss: 0.0232 - val_accuracy: 0.9829 - val_loss: 0.0570 - learning_rate: 1.0000e-05
Epoch 4/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 159s 134ms/step - accuracy: 0.9929 - loss: 0.0212 - val_accuracy: 0.9823 - val_loss: 0.0536 - learning_rate: 1.0000e-05
Epoch 5/5
1094/1094 ━━━━━━━━━━━━━━━━━━━━ 170s 145ms/step - accuracy: 0.9944 - loss: 0.0176 - val_accuracy: 0.9834 - val_loss: 0.0530 - learning_rate: 1.0000e-05
Restoring model weights from the end of the best epoch: 5.


In [ ]:
# Evaluate model on the test dataset
test_loss, test_acc = model1.evaluate(test_ds)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")

239/239 ━━━━━━━━━━━━━━━━━━━━ 40s 167ms/step - accuracy: 0.9849 - loss: 0.0510
Test Loss: 0.0615
Test Accuracy: 0.9821


In [ ]:
train_ds_for_class = tf.keras.utils.image_dataset_from_directory(
    dataset_dir + "train",
    seed=42,
    image_size=image_size,
    batch_size=batch_size
)

# Save the class names
class_names = train_ds_for_class.class_names
print("Class names:", class_names)

Found 69953 files belonging to 51 classes.
Class names: ['Apple Apple scab', 'Apple Black rot', 'Apple Cedar apple rust', 'Apple healthy', 'Bacterial leaf blight in rice leaf', 'Blight in corn Leaf', 'Blueberry healthy', 'Brown spot in rice leaf', 'Cercospora leaf spot', 'Cherry (including sour) Powdery mildew', 'Cherry (including_sour) healthy', 'Common Rust in corn Leaf', 'Corn (maize) healthy', 'Garlic', 'Grape Black rot', 'Grape Esca Black Measles', 'Grape Leaf blight Isariopsis Leaf Spot', 'Grape healthy', 'Gray Leaf Spot in corn Leaf', 'Leaf smut in rice leaf', 'Orange Haunglongbing Citrus greening', 'Peach healthy', 'Pepper bell Bacterial spot', 'Pepper bell healthy', 'Potato Early blight', 'Potato Late blight', 'Potato healthy', 'Raspberry healthy', 'Soybean healthy', 'Strawberry Leaf scorch', 'Strawberry healthy', 'Tomato Bacterial spot', 'Tomato Early blight', 'Tomato Late blight', 'Tomato Leaf Mold', 'Tomato Septoria leaf spot', 'Tomato Spider mites Two spotted spider mite',

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

# -------------------------------
# 1️⃣ Path to the single image
# -------------------------------
img_path = "/content/dataset/dataset/train/Bacterial leaf blight in rice leaf/aug_0_0.jpg"

# -------------------------------
# 2️⃣ Load and preprocess the image
# -------------------------------
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)  # shape becomes (1, 224, 224, 3)
x = preprocess_input(x)        # preprocess for MobileNetV2

# -------------------------------
# 3️⃣ Predict
# -------------------------------
preds = model1.predict(x)
pred_idx = np.argmax(preds[0])
pred_class = class_names[pred_idx]

print(f"Predicted class: {pred_class}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted class: Bacterial leaf blight in rice leaf


In [ ]:
with open("class_labels.txt", "w") as f:
    for class_name in class_names:
        f.write(class_name + "\n")


In [ ]:
import json

class_dict = {i: name for i, name in enumerate(class_names)}
with open("class_labels.json", "w") as f:
    json.dump(class_dict, f, indent=2)

In [ ]:
# Save model in Keras 3 format
model1.save("/content/mobilenetv2_51classes.keras")
print("Model saved in Keras native format (.keras)!")

Model saved in Keras native format (.keras)!


In [ ]:
model1.save("/content/mobilenetv2_51classes.h5")
print("Model saved in HDF5 (.h5) format!")

Model saved in HDF5 (.h5) format!


In [ ]:
import shutil

# Path to your model folder
model_folder = "/content/mobilenetv2_51classes_savedmodel"
zip_file = "/content/mobilenetv2_51classes_savedmodel.zip"

# Zip the folder
shutil.make_archive(base_name=zip_file.replace(".zip",""), format='zip', root_dir=model_folder)
print(f"Folder zipped at: {zip_file}")


Folder zipped at: /content/mobilenetv2_51classes_savedmodel.zip


In [ ]:
import tensorflow as tf

saved_model_dir = "/content/mobilenetv2_51classes_savedmodel"
tflite_model_file = "/content/mobilenetv2_51classes.tflite"

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
tflite_model = converter.convert()

# Save TFLite model
with open(tflite_model_file, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved at: {tflite_model_file}")

TFLite model saved at: /content/mobilenetv2_51classes.tflite


In [ ]:
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

with open("/content/mobilenetv2_51classes_quant.tflite", "wb") as f:
    f.write(tflite_quant_model)

print("Quantized TFLite model saved!")


Quantized TFLite model saved!
